In [1]:
library(data.table)
library(dplyr)
library(tidyr)
library(feather)

# Load Data ---------------------------------------------------------------
path <- "/media/tin/DATA/Kaggle/Instacart/Input/"

aisles <- read_feather(file.path(path, "aisles.feather"))
departments <- read_feather(file.path(path, "departments.feather"))
orderp <- read_feather(file.path(path, "order_products__prior.feather"))
ordert <- read_feather(file.path(path, "order_products__train.feather"))
orders <- read_feather(file.path(path, "orders.feather"))
products <- read_feather(file.path(path, "products.feather"))

# user --> order --> product

------------------------------------------------------------------------------
data.table + dplyr code now lives in dtplyr.
Please library(dtplyr)!
------------------------------------------------------------------------------

Attaching package: ‘dplyr’

The following objects are masked from ‘package:data.table’:

    between, first, last

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [ ]:
# control + /
# aisles <- fread(file.path(path, "aisles.csv"))
# departments <- fread(file.path(path, "departments.csv"))
# orderp <- fread(file.path(path, "order_products__prior.csv"))
# ordert <- fread(file.path(path, "order_products__train.csv"))
# orders <- fread(file.path(path, "orders.csv"))
# products <- fread(file.path(path, "products.csv"))


# write_feather(aisles, file.path(path, "aisles.feather"))
# write_feather(departments, file.path(path, "departments.feather"))
# write_feather(orderp, file.path(path, "order_products__prior.feather"))
# write_feather(ordert , file.path(path, "order_products__train.feather"))
# write_feather(orders, file.path(path, "orders.feather"))
# write_feather(products, file.path(path, "products.feather"))

In [2]:
# Reshape data ------------------------------------------------------------
aisles$aisle <- as.factor(aisles$aisle)
departments$department <- as.factor(departments$department)
orders$eval_set <- as.factor(orders$eval_set)
products$product_name <- as.factor(products$product_name)

products <- products %>% 
  inner_join(aisles) %>% inner_join(departments) %>% 
  select(-aisle_id, -department_id)
rm(aisles, departments)

#create new column user_id in ordert, matching correctly customer and his order
ordert$user_id <- orders$user_id[match(ordert$order_id, orders$order_id)]

orders_products <- orders %>% inner_join(orderp, by = "order_id")

rm(orderp)
gc()

Joining, by = "aisle_id"
Joining, by = "department_id"


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,625818,33.5,1168576,62.5,1083715,57.9
Vcells,196989445,1503.0,330772829,2523.6,261880439,1998.0


In [ ]:
# head(ordert, 5)
# head(orders, 5)
# head(orderp, 5)

In [3]:
# Products ----------------------------------------------------------------
prd <- orders_products %>%
  arrange(user_id, order_number, product_id) %>%
  group_by(user_id, product_id) %>%
  mutate(product_time = row_number()) %>% #row_number: ranking, counter for times purchased; go with previous group_by
  ungroup() %>%
  group_by(product_id) %>% #by products, aggragate over all orders and all customers
  summarise(
    prod_orders = n(),
    prod_reorders = sum(reordered),
    prod_first_orders = sum(product_time == 1),
    prod_second_orders = sum(product_time == 2)
  )

prd$prod_reorder_probability <- prd$prod_second_orders / prd$prod_first_orders
prd$prod_reorder_times <- 1 + prd$prod_reorders / prd$prod_first_orders
prd$prod_reorder_ratio <- prd$prod_reorders / prd$prod_orders

prd <- prd %>% select(-prod_reorders, -prod_first_orders, -prod_second_orders)

rm(products)
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,582204,31.1,16708522,892.4,20885653,1115.5
Vcells,196769045,1501.3,569524340,4345.2,479645902,3659.5


In [27]:
prd[1:300000,]

product_id,prod_orders,prod_reorder_probability,prod_reorder_times,prod_reorder_ratio
1,1852,0.38547486,2.586592,0.6133909
2,90,0.10256410,1.153846,0.1333333
3,277,0.48648649,3.743243,0.7328520
4,329,0.35164835,1.807692,0.4468085
5,15,0.66666667,2.500000,0.6000000
6,8,0.40000000,1.600000,0.3750000
7,30,0.33333333,1.666667,0.4000000
8,165,0.36585366,2.012195,0.5030303
9,156,0.41891892,2.108108,0.5256410
10,2572,0.31466877,2.028391,0.5069984


In [4]:
# Users -------------------------------------------------------------------
users <- orders %>%
  filter(eval_set == "prior") %>%
  group_by(user_id) %>%
  summarise(
    user_orders = max(order_number),
    user_period = sum(days_since_prior_order, na.rm = T),
    user_mean_days_since_prior = mean(days_since_prior_order, na.rm = T)
  )

us <- orders_products %>%
  group_by(user_id) %>%
  summarise(
    user_total_products = n(),
    user_reorder_ratio = sum(reordered == 1) / sum(order_number > 1),
    user_distinct_products = n_distinct(product_id)
  )

users <- users %>% inner_join(us)
users$user_average_basket <- users$user_total_products / users$user_orders

us <- orders %>%
  filter(eval_set != "prior") %>%
  select(user_id, order_id, eval_set,
         time_since_last_order = days_since_prior_order)

users <- users %>% inner_join(us)

rm(us)
gc()

Joining, by = "user_id"
Joining, by = "user_id"


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,582901,31.2,13366817,713.9,20885653,1115.5
Vcells,198420220,1513.9,569524340,4345.2,479645902,3659.5


In [30]:
users[1:50000,]

user_id,user_orders,user_period,user_mean_days_since_prior,user_total_products,user_reorder_ratio,user_distinct_products,user_average_basket,order_id,eval_set,time_since_last_order
1,10,176,19.555556,59,0.75925926,18,5.900000,1187899,train,14
2,14,198,15.230769,195,0.51098901,102,13.928571,1492625,train,30
3,12,133,12.090909,88,0.70512821,33,7.333333,2774568,test,11
4,5,55,13.750000,18,0.07142857,17,3.600000,329954,test,30
5,4,40,13.333333,37,0.53846154,23,9.250000,2196797,train,6
6,3,18,9.000000,14,0.20000000,12,4.666667,1528013,test,22
7,20,203,10.684211,206,0.71134021,68,10.300000,525192,train,6
8,3,60,30.000000,49,0.46428571,36,16.333333,880375,train,10
9,3,36,18.000000,76,0.39130435,58,25.333333,1094988,train,30
10,5,79,19.750000,143,0.35507246,94,28.600000,1822501,train,30


In [5]:
# Database ----------------------------------------------------------------
data <- orders_products %>%
  group_by(user_id, product_id) %>% 
  summarise(
    up_orders = n(),
    up_first_order = min(order_number),
    up_last_order = max(order_number),
    up_average_cart_position = mean(add_to_cart_order)) #analyse the sequence of purchased product

rm(orders_products, orders)

data <- data %>% 
  inner_join(prd, by = "product_id") %>%
  inner_join(users, by = "user_id")

data$up_order_rate <- data$up_orders / data$user_orders
data$up_orders_since_last_order <- data$user_orders - data$up_last_order
data$up_order_rate_since_first_order <- data$up_orders / (data$user_orders - data$up_first_order + 1)

data <- data %>% 
  left_join(ordert %>% select(user_id, product_id, reordered), 
            by = c("user_id", "product_id"))

rm(ordert, prd, users)
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,582960,31.2,9900141,528.8,20885653,1115.5
Vcells,233925384,1784.8,683509208,5214.8,565281032,4312.8


In [6]:
data[1:300000,]

user_id,product_id,up_orders,up_first_order,up_last_order,up_average_cart_position,prod_orders,prod_reorder_probability,prod_reorder_times,prod_reorder_ratio,⋯,user_reorder_ratio,user_distinct_products,user_average_basket,order_id,eval_set,time_since_last_order,up_order_rate,up_orders_since_last_order,up_order_rate_since_first_order,reordered
1,196,10,1,10,1.400000,35791,0.5825000,4.473875,0.7764801,⋯,0.7592593,18,5.90000,1187899,train,14,1.00000000,0,1.0000000,1
1,10258,9,2,10,3.333333,1946,0.5529623,3.493716,0.7137718,⋯,0.7592593,18,5.90000,1187899,train,14,0.90000000,0,1.0000000,1
1,10326,1,5,5,5.000000,5526,0.5215809,2.873635,0.6520087,⋯,0.7592593,18,5.90000,1187899,train,14,0.10000000,5,0.1666667,NA
1,12427,10,1,10,3.300000,6476,0.5294818,3.857058,0.7407350,⋯,0.7592593,18,5.90000,1187899,train,14,1.00000000,0,1.0000000,NA
1,13032,3,2,10,6.333333,3751,0.4797823,2.916796,0.6571581,⋯,0.7592593,18,5.90000,1187899,train,14,0.30000000,0,0.3333333,1
1,13176,2,2,5,6.000000,379450,0.7118844,5.972111,0.8325550,⋯,0.7592593,18,5.90000,1187899,train,14,0.20000000,5,0.2222222,NA
1,14084,1,1,1,2.000000,15935,0.6291501,5.290505,0.8109821,⋯,0.7592593,18,5.90000,1187899,train,14,0.10000000,9,0.1000000,NA
1,17122,1,5,5,6.000000,13880,0.5274262,3.082390,0.6755764,⋯,0.7592593,18,5.90000,1187899,train,14,0.10000000,5,0.1666667,NA
1,25133,8,3,10,4.000000,6196,0.5807453,3.848447,0.7401549,⋯,0.7592593,18,5.90000,1187899,train,14,0.80000000,0,1.0000000,1
1,26088,2,1,2,4.500000,2523,0.4049871,2.169390,0.5390408,⋯,0.7592593,18,5.90000,1187899,train,14,0.20000000,8,0.2000000,1


In [7]:
glimpse(data)

#ahahahaha
#list down every products in every order of every person in train and test
#check status reordered or not
#13 mil products
#if reordered predict positive in test set, combine them for each order and submit

Observations: 13,307,953
Variables: 24
$ user_id                         <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...
$ product_id                      <int> 196, 10258, 10326, 12427, 13032, 13...
$ up_orders                       <int> 10, 9, 1, 10, 3, 2, 1, 1, 8, 2, 2, ...
$ up_first_order                  <int> 1, 2, 5, 1, 2, 2, 1, 5, 3, 1, 1, 3,...
$ up_last_order                   <int> 10, 10, 5, 10, 10, 5, 1, 5, 10, 2, ...
$ up_average_cart_position        <dbl> 1.400000, 3.333333, 5.000000, 3.300...
$ prod_orders                     <int> 35791, 1946, 5526, 6476, 3751, 3794...
$ prod_reorder_probability        <dbl> 0.5825000, 0.5529623, 0.5215809, 0....
$ prod_reorder_times              <dbl> 4.473875, 3.493716, 2.873635, 3.857...
$ prod_reorder_ratio              <dbl> 0.7764801, 0.7137718, 0.6520087, 0....
$ user_orders                     <int> 10, 10, 10, 10, 10, 10, 10, 10, 10,...
$ user_period                     <dbl> 176, 176, 176, 176, 176, 176, 176, ...
$ user_mean_d

In [ ]:
# Train / Test datasets ---------------------------------------------------
train <- as.data.frame(data[data$eval_set == "train",])
train$eval_set <- NULL
train$user_id <- NULL
train$product_id <- NULL
train$order_id <- NULL
train$reordered[is.na(train$reordered)] <- 0

test <- as.data.frame(data[data$eval_set == "test",])
test$eval_set <- NULL
test$user_id <- NULL
test$reordered <- NULL

rm(data)
gc()

In [ ]:
train

In [ ]:
test

In [ ]:
path <- path <- "/media/tin/DATA/Kaggle/Instacart/Input/"
write.csv(train, file = file.path(path,"train_prepared.csv"), row.names = F)
write.csv(test, file = file.path(path,"test_prepared.csv"), row.names = F)

In [ ]:
path <- "/media/tin/DATA/Kaggle/Instacart/Input/train_prepared.feather"
write_feather(train, path)

In [ ]:
path <- "/media/tin/DATA/Kaggle/Instacart/Input/test_prepared.feather"
write_feather(test, path)